# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


## TD7 : Mélanges de lois


Ce TD est composé de trois exercices. Le premier concerne l'utilisation du mélange de lois normales pour effectuer une classification. Le deuxième exercice concerne la génération d'un échantillon aléatoire d'un mélange de deux lois exponentielles et l'estimation de ses paramètres avec l'échantillonnage de Gibbs. L'estimation des paramètres avec l'algorithme EM constitue l'objet du troisième exercice.

In [ ]:
using CSV, DataFrames, Statistics, Distributions, Gadfly
include("functions.jl");

## Exercice 1 - Salaires des diplômés en fonction du type d'université

Dans cet exercice, nous étudierons le salaire médian des diplomés au États-Unis en fonction du type d'université :

- Engineering
- Ivy League 
- Liberal Arts
- Party    
- State

Les données proviennent de *PayScale Inc.* et ont été récupérées sur le site web du *Wall Street Journal* :
http://online.wsj.com/public/resources/documents/info-Salaries_for_Colleges_by_Type-sort.html.

In [ ]:
# Chargement des données

data = CSV.read("salaries.csv")
first(data,5)

In [ ]:
# Affichage du nom des colonnes du DataFrame
names(data)

In [ ]:
# Transformation des salaires en milliers de dollars pour un affichage plus élégant
for i=3:8
    data[!,i] = data[:,i]/1000
end

### (a) Illustrez l'histogramme des salaires médians des diplomés en début de carrière

Pour que votre graphique soit plus élégant, vous pouvez afficher le salaire médian en milliers de dollars (k$). Le nombre de classe de l'histogramme devrait être environ la racine carré du nombre d'observations.

### (b) Illustrez les diagrammes en boîtes salaires médians des diplomés en début de carrière en fonction des types d'école

Avec la librairie Gadfly, utilisez la géométrie Geom.boxplot pour illustrer des diagrammes en boîte.

### (c) Estimez les paramètres d'un mélange de deux lois normales avec l'algorithme EM

L'algorithme EM pour un mélange de deux lois normales est fourni dans le fichier *functions.jl*. Il s'agit de la fonction `GMMemfit(y::Vector{<:Real})`. La sortie de cette fonction donne un objet de type *MixtureModel* qui s'utilise avec les méthodes standards (`pdf`, `logpdf`, etc.) de la librairie *Distributions*. Les méthodes suivantes sont particulières au type *MixtureModel*:
- `probs(fd::MixtureModel)` : retourne le vecteur des poids de chacune des composantes ;
- `component(fd::MixtureModel, k)` : retourne un objet de type *Distribution* correspondant à la composante k.

In [ ]:
# Permutation des composantes pour que la deuxième composante possède la plus grande moyenne

if mean(component(fd, 2)) < mean(component(fd, 1))
    f₀ = component(fd, 2)
    f₁ = component(fd, 1)
    ω = probs(fd)[1]
    
    fd = GMM(ω, mean(f₀), std(f₀), mean(f₁), std(f₁))
end

### (d) Tracez les densités des deux composantes obtenues ainsi que la densité du mélange

Tracez ces densités pour les salaires médians de 30 à 50 k$. 

Pour un affichage simplifié, vous pouvez créer un DataFrame avec une colonne pour $x$, une colonne pour le première composante f₀, une colonne pour la deuxième composante f₁ et une colonne pour le mélange f. Ensuite, vous pouvez utiliser la fonction `melt(df, :x)` pour transformer les colonnes de votre DataFrame en variables utiles pour l'affichage.

In [ ]:
x = 30:.1:70




### (e) Estimez pour chacun des salaires médians la probabilité qu'il provienne de la composante de plus grande moyenne

### (f) Combien d'universités de chaque type se retrouve dans la catégorie correspondante à la composante ayant  la moyenne la plus élevée pour les salaires médians.

Supposez que les observations pour lesquelles p>1/2 proviennent de la composante avec la plus grande moyenne.

[**Optionel**]Pour dénombrer par type, vous pouvez utilisez la fonction *by()* de la librairie *DataFrame*.

### (g) Combien d'universités de chaque type se retrouve dans la catégorie correspondante à la composante ayant  la moyenne la plus basse pour les salaires médians.

### (h) Quelles sont les universités d'état qui se comparent aux grandes universités ?

### (i) Quelles sont les universités d'ingénierie qui se comparent aux universités d'état ?

## Exercice 2 :  Mélange de deux lois exponentielles

Soit le mélange de deux lois exponentielles suivant :

$$f_{(Y|\boldsymbol{\theta})}(y) = (1- \omega) ~ \mathcal{E}xp(\lambda_0) + \omega ~ \mathcal{E}xp(\lambda_1).$$

avec $\boldsymbol{\theta} = (\omega, \lambda_0, \lambda_1)$. Attention, on utilise ici la seule forme acceptable de la loi exponentielle qui stipule que si $Y \sim \mathcal{E}xp(\lambda)$, alors $\mathbb{E}(Y) = 1/\lambda$ (voir par exemple https://en.wikipedia.org/wiki/Exponential_distribution ).


### (a) Générez un échantillon aléatoire d'un mélange de lois exponentielles

Générez un échantillon aléatoire de taille $n = 1000$ du mélange de lois exponentielles de paramètres $\boldsymbol{\theta} = (1/5, 1, 1/4)$.

Pour ce faire, rappelez-vous de l'interprétation de $\omega$ et de la variable latente $Z$.

In [ ]:
n = 1000
ω = 1/5
λ₀ = 1
λ₁ = 1/4



### (b) Tracez l'histogramme de l'échantillon généré

### (c) Générez un échantillon de la loi *a posteriori* des paramètres en utilisant l'échantillonnage de Gibbs

Utilisez la loi *a priori* impropre suivante pour les paramètres :
$$f_{\boldsymbol{\theta}}(\boldsymbol{\theta}) \propto (1-\omega)^{-1}\omega^{-1} \times \lambda_0^{-1} \times \lambda_1^{-1}.$$

Faîtes au moins 1000 itérations pour l'échantillonnage de Gibbs. Tracez les chaînes marginales générées.

In [ ]:
# Initialisation des vecteur pour le MCMC
niter = 1000
ω = zeros(niter)
λ₀ = zeros(niter)
λ₁ = zeros(niter)
z = falses(n,niter)

# Valeurs initiales
ω[1] = .5
λ₀[1] = 1
λ₁[1] = 1



# Stockage dans un DataFrame pour un affichage rapide
# df = DataFrame(Itération = 1:niter, ω = ω, λ₀ = λ₀, λ₁ = λ₁)
# first(df,5)

### (d) Estimez les paramètres avec l'algorithme EM

Vous devez trouvez les expressions des estimations améliorées pour le mélange de lois exponentielles avec l'algorithme EM.